In [141]:
import pandas as pd
import ast
import numpy as np

In [142]:
path = 'if_else.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [143]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [144]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,"Performers dressed as a princess, deer, table,...",The deer and table are protecting the princess...,"Premise: Performers dressed as a princess, dee...",1. The correct answer is **neutral**. The hypo...,1. the correct answer is **neutral**. the hypo...


In [145]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [146]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [147]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [116]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)
paper

,premise,hypothesis,label,llm_label,rationale,input
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral,neutral,The person could be training his horse for a c...,A person on a horse jumps over a broken down a...
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction,contradiction,The person on the horse is not at the diner.,A person on a horse jumps over a broken down a...
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment,entailment,The person is outdoors because the airplane is...,A person on a horse jumps over a broken down a...
3,Children smiling and waving at camera,They are smiling at their parents,neutral,neutral,"They are smiling at the camera, not their pare...",Children smiling and waving at camera</s>They ...
4,Children smiling and waving at camera,There are children present,entailment,entailment,The children are present because they are smil...,Children smiling and waving at camera</s>There...
...,...,...,...,...,...,...
9995,A baby in a white and light purple outfit crying.,A baby is crying because it is hungry.,neutral,neutral,"The baby is crying, but it is not necessarily ...",A baby in a white and light purple outfit cryi...
9996,Baby in a purple onesie crying.,The baby is sleeping peacefully.,contradiction,contradiction,"The baby is crying, not sleeping.",Baby in a purple onesie crying.</s>The baby is...
9997,Baby in a purple onesie crying.,The child is under three years old.,entailment,entailment,The child is a baby.,Baby in a purple onesie crying.</s>The child i...
9998,Baby in a purple onesie crying.,The baby is a girl.,neutral,neutral,The baby is wearing a purple onesie.,Baby in a purple onesie crying.</s>The baby is...


In [117]:
# paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index(['premise', 'hypothesis'], inplace=True)
# paper.index = paper.index.str.strip()

In [118]:
df.set_index(['premise', 'hypothesis'], inplace=True)

In [119]:
paper['my_label'] = df['LLM_answer']

In [120]:
sum(paper.label == paper.my_label) / len(paper)

0.8672

In [121]:
df.to_csv('consensus - full.csv')

In [124]:
import pandas as pd
TYPE = 'consensus'
temp = pd.read_csv(f'{TYPE} - full.csv')

import re
temp[~temp['rationale'].apply(lambda x: bool(re.match(r'\d+\.', x.strip())))]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
